# Diversity Testbench

---
---

## Ember Package Testing (WIP)

### Setup Dependencies

In [10]:
import logging, sys, os
from typing import Dict, Any, List

In [11]:
os.environ["OPENAI_API_KEY"] = ''

In [12]:
openai_key = os.getenv("OPENAI_API_KEY")

In [13]:
# fixing dependencies if current path is <root>/src/ember/examples/diversity_testbench.ipynb
target_dir = 'src/ember/examples'
if os.getcwd()[-18:] == target_dir:
    os.chdir('../../..')
print(os.getcwd())

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

/Users/kathleenge/Desktop/NON/ember-v2


In [14]:
!echo $PWD

/Users/kathleenge/desktop/non/ember-v2


NOTE: things below this are to install required dependencies (only do this the venv)

In [15]:
# %pip install -q -e .
# %pip install -q google-generativeai==0.7.2

### Ember Repo Loads (WIP)

In [16]:
# from ember.core.registry.model.model_module.lm import LMModule, LMModuleConfig
from ember.core.registry.model.config.settings import initialize_ember
from ember.core.registry.model.base.services.model_service import ModelService
from ember.core.registry.model.base.schemas.model_info import ModelInfo
from ember.core.registry.model.base.schemas.cost import ModelCost, RateLimit
from ember.core.registry.model.base.schemas.provider_info import ProviderInfo

from ember.core.registry.model import load_model, ChatResponse
from ember.core.registry.model.base.services.model_service import ModelService


In [17]:
model_registry = initialize_ember()
print(model_registry.list_models())
llm = ModelService(registry=model_registry)

ValidationError: 4 validation errors for EmberSettings
registry.models.3.api_key
  Value error, No API key provided or defaulted. [type=value_error, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
registry.models.4.api_key
  Value error, No API key provided or defaulted. [type=value_error, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
registry.models.5.api_key
  Value error, No API key provided or defaulted. [type=value_error, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
registry.models.6.api_key
  Value error, No API key provided or defaulted. [type=value_error, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [63]:
model_registry.list_models()

['openai:gpt-4o',
 'openai:gpt-4o-mini',
 'openai:o1',
 'anthropic:claude-3.5-sonnet',
 'google:gemini-1.5-pro',
 'google:gemini-2.0-flash',
 'google:gemini-exp-1206']

In [64]:
model_ids: List[str] = [
            "openai:o1",
            "openai:gpt-4o",
            "openai:gpt-4o-mini",
            # "anthropic:claude-3.5-sonnet", # API key not working
            # "invalid:model",  # Expected to trigger an error.
            # "google:model/gemini-1.5-pro", # need to fix model alignment
        ]

In [65]:
for model_id in model_ids:
            try:
                print(f"➡️ Testing model: {model_id}")

                # Two usage styles are demonstrated below:
                # 1. Service-based invocation: Recommended for automatic usage tracking.
                service_response: ChatResponse = llm.invoke_model(
                    model_id=model_id,
                    prompt="Explain quantum computing in 50 words",
                )
                print(f"🛎️ Service response from {model_id}:\n{service_response.data}\n")

                # 2. Direct model instance usage: Useful for more granular or PyTorch-like workflows.
                model = load_model(model_id=model_id, registry=model_registry)
                direct_response: ChatResponse = model(
                    prompt="What's the capital of France?"
                )
                print(f"🎯 Direct response from {model_id}:\n{direct_response.data}\n")

            except Exception as error:
                print(f"❌ Error with model {model_id}: {str(error)}")
                continue


➡️ Testing model: openai:o1
🛎️ Service response from openai:o1:
Quantum computing uses quantum bits, or qubits, which can exist in superpositions of states and become entangled. These properties enable massive parallel processing, making certain computations exponentially faster than classical computers. By harnessing quantum phenomena, quantum computing aims to solve complex problems, including cryptography, optimization, and simulations of quantum systems.

🎯 Direct response from openai:o1:
The capital of France is Paris.

➡️ Testing model: openai:gpt-4o
🛎️ Service response from openai:gpt-4o:
Quantum computing harnesses quantum mechanics to process information using qubits, which can exist in multiple states simultaneously. This allows for potentially exponential increases in computational power compared to classical computers, enabling the solving of complex problems much more efficiently, such as cryptography, optimization, and simulations of quantum systems.

🎯 Direct response fr

In [66]:
# Register an OpenAI GPT-4o model
# openai_info = ModelInfo(
#     model_id="openai:gpt-4o",
#     model_name="gpt-4o",
#     cost=ModelCost(input_cost_per_thousand=0.03, output_cost_per_thousand=0.06),
#     rate_limit=RateLimit(tokens_per_minute=80000, requests_per_minute=5000),
#     provider=ProviderInfo(name="OpenAI", default_api_key=openai_key),
#     api_key=openai_key,
# )
# model_registry.register_model(openai_info)

In [67]:
response = llm(prompt="Hello!", model_id="openai:o1")
print(response.data)

Hello there! How can I help you today?


---
---

## Neural Similarity Scoring - Cosine Similarity (WIP)

- from `src/ember/core/utils/embedding_utils.py`
- from jason
- need to merge

In [68]:
%pip install -q openai

Note: you may need to restart the kernel to use updated packages.


In [7]:
from __future__ import annotations

from abc import ABC, abstractmethod
from typing import List, Protocol
import math

import openai
import os


################################################################
# 1) Embedding Model Interfaces & Implementations
################################################################


class EmbeddingModel(Protocol):
    """Interface for embedding models.

    This protocol defines the minimal interface required to compute a text
    embedding. Implementations may use local models, external APIs, or custom
    neural networks.

    Methods:
        embed_text: Compute the embedding for a given text.
    """

    def embed_text(self, text: str) -> List[float]:
        """Computes the embedding vector for the provided text.

        Args:
            text (str): The text to be embedded.

        Returns:
            List[float]: A list of floats representing the embedding vector.
        """
        ...

class Text_Embedding_3_EmbeddingModel(Protocol):
    """Interface for embedding models.

    This protocol defines the minimal interface required to compute a text
    embedding. Implementations may use local models, external APIs, or custom
    neural networks.

    Methods:
        embed_text: Compute the embedding for a given text.
    """

    def __init__(self, api_key: str = None):
        """Initializes the embedding model with the OpenAI API key.

        Args:
            api_key (str): OpenAI API key for authentication.
        """
        self.api_key = api_key or os.environ.get("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OpenAI API key must be provided or set in the environment variable OPENAI_API_KEY.")
        openai.api_key = self.api_key

    def embed_text(self, text: str) -> List[float]:
        """Computes the embedding vector for the provided text.

        Args:
            text (str): The text to be embedded.

        Returns:
            List[float]: A list of floats representing the embedding vector.
        """
        response = openai.Embedding.create(
            model="text-embedding-3",
            input=text
        )
        return response["data"][0]["embedding"]


class MockEmbeddingModel:
    """Mock implementation of an embedding model using naive ASCII encoding.

    This simple model converts each character in the text to a normalized ASCII
    value. It is intended solely for demonstration and testing purposes.

    Methods:
        embed_text: Converts text to a sequence of normalized ASCII values.
    """

    def embed_text(self, text: str) -> List[float]:
        """Embeds text by converting each character to its normalized ASCII code.

        Args:
            text (str): The input text to be embedded.

        Returns:
            List[float]: A list of floats representing the embedding. Returns an
            empty list if the text is empty.
        """
        if not text:
            return []
        return [ord(ch) / 256.0 for ch in text]


################################################################
# 2) Similarity Metric Interface & Implementations
################################################################


class SimilarityMetric(ABC):
    """Abstract base class for computing similarity between embedding vectors.

    Subclasses must implement the similarity method to calculate a similarity
    score between two vectors.
    """

    @abstractmethod
    def similarity(self, vec_a: List[float], vec_b: List[float]) -> float:
        """Calculates the similarity between two embedding vectors.

        Args:
            vec_a (List[float]): The first embedding vector.
            vec_b (List[float]): The second embedding vector.

        Returns:
            float: The similarity score, typically in the range [0, 1] or [-1, 1].
        """
        ...


class CosineSimilarity(SimilarityMetric):
    """Implementation of cosine similarity for embedding vectors.

    The cosine similarity is defined as:
        similarity(a, b) = (a · b) / (||a|| * ||b||)

    Returns 0.0 if either vector is empty or if any vector's norm is zero.
    """

    def similarity(self, vec_a: List[float], vec_b: List[float]) -> float:
        """Computes cosine similarity between two embedding vectors.

        Args:
            vec_a (List[float]): The first embedding vector.
            vec_b (List[float]): The second embedding vector.

        Returns:
            float: The cosine similarity score.
        """
        if not vec_a or not vec_b:
            return 0.0

        dot_product: float = sum(a * b for a, b in zip(vec_a, vec_b))
        norm_a: float = math.sqrt(sum(a * a for a in vec_a))
        norm_b: float = math.sqrt(sum(b * b for b in vec_b))
        if norm_a == 0 or norm_b == 0:
            return 0.0

        return dot_product / (norm_a * norm_b)


################################################################
# 3) High-Level Utility Function
################################################################


def calculate_text_similarity(
    text1: str, text2: str, model: EmbeddingModel, metric: SimilarityMetric
) -> float:
    """Calculates text similarity using an embedding model and a similarity metric.

    This function generates embeddings for the provided texts and then computes a
    similarity score using the given similarity metric.

    Args:
        text1 (str): The first text string.
        text2 (str): The second text string.
        model (EmbeddingModel): An instance conforming to the embedding model interface.
        metric (SimilarityMetric): An instance implementing a similarity metric.

    Returns:
        float: The computed similarity score.
    """
    embedding1: List[float] = model.embed_text(text=text1)
    embedding2: List[float] = model.embed_text(text=text2)
    return metric.similarity(vec_a=embedding1, vec_b=embedding2)

In [8]:
mock_model: MockEmbeddingModel = MockEmbeddingModel()
cosine: CosineSimilarity = CosineSimilarity()

text_a: str = "Hello world!"
text_b: str = "Hello, world??"

score: float = calculate_text_similarity(
    text1=text_a, text2=text_b, model=mock_model, metric=cosine
)
print(f"Similarity between '{text_a}' and '{text_b}': {score}")

NameError: name 'mock_model' is not defined

---
---

## Compression Ratio (WIP)

from `src/ember/core/utils/eval/evaluators.py`

In [71]:
%pip install -q diversity==0.2.0
%pip install -q spacy==3.8.4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [72]:
from __future__ import annotations

import re
import subprocess
from typing import Any, Dict, TypeVar, Optional, List, Generic, Callable, Union

from ember.core.utils.eval.base_evaluator import IEvaluator, EvaluationResult
from ember.core.utils.eval.extractors import RegexExtractor

from diversity import compression_ratio

T_out = TypeVar("T_out")
T_truth = TypeVar("T_truth")


class ComposedEvaluator(IEvaluator[T_out, T_truth], Generic[T_out, T_truth]):
    """Combines an output extractor with an evaluator for the extracted data.

    This evaluator first transforms the system output using the provided extractor,
    then evaluates the extracted value using the specified base evaluator.

    Args:
        extractor: An object with an `extract` method to process the system output.
        base_evaluator (IEvaluator): An evaluator that processes the extracted output.

    Returns:
        EvaluationResult: The result of the evaluation.
    """

    def __init__(
        self,
        extractor: Any,  # Expecting an extractor with an `extract` method.
        base_evaluator: IEvaluator[Any, Any],
    ) -> None:
        self.extractor = extractor
        self.base_evaluator = base_evaluator

    def evaluate(
        self, system_output: T_out, correct_answer: Any, **kwargs: Any
    ) -> EvaluationResult:
        """Evaluates the provided system output against the correct answer.

        Args:
            system_output (T_out): The raw output generated by the system.
            correct_answer (Any): The expected correct answer.
            **kwargs: Additional keyword arguments for extraction or evaluation.

        Returns:
            EvaluationResult: The result of evaluating the extracted value.
        """
        extracted_value = self.extractor.extract(system_output, **kwargs)
        return self.base_evaluator.evaluate(extracted_value, correct_answer, **kwargs)


# Basic Evaluators


class ExactMatchEvaluator(IEvaluator[str, str]):
    """Evaluator to check for an exact match between two strings,
    ignoring differences in whitespace and case.

    Example:
        evaluator = ExactMatchEvaluator()
        result = evaluator.evaluate("Hello World", "hello   world")

    Args:
        compare_fn (Optional[Callable[[str, str], bool]]): Optional custom comparison function.
            If not provided, strings are normalized (whitespace removed, lowercase) before comparison.

    Returns:
        EvaluationResult: The result containing a correctness flag and a score.
    """

    def __init__(self, compare_fn: Optional[Callable[[str, str], bool]] = None) -> None:
        self.compare_fn = compare_fn or self._default_compare

    def _default_compare(self, str1: str, str2: str) -> bool:
        """Default string comparison function that ignores case and whitespace.

        Args:
            str1 (str): First string to compare
            str2 (str): Second string to compare

        Returns:
            bool: True if strings match after normalization
        """
        return str1.strip().lower() == str2.strip().lower()

    def evaluate(
        self, system_output: str, correct_answer: str, **kwargs: Any
    ) -> EvaluationResult:
        """Evaluates whether a system output exactly matches the correct answer.

        Args:
            system_output (str): The system-generated string.
            correct_answer (str): The expected answer string.
            **kwargs: Additional keyword arguments (unused).

        Returns:
            EvaluationResult: An object with `is_correct` set to True if the normalized strings match,
                              along with a corresponding score.
        """
        is_correct = self.compare_fn(system_output, correct_answer)
        score = 1.0 if is_correct else 0.0
        return EvaluationResult(is_correct=is_correct, score=score)

class DiversityScoringEvaluator(IEvaluator[List[str], None]):
    """
    Evaluator to test ensemble outputs -> score them (float)
    """
    def evaluate(
            self, 
            system_output: List[str], 
            **kwargs) -> EvaluationResult:
        if system_output is None or len(system_output) == 0:
            return EvaluationResult(is_correct=False, score=-1)

        # current compression ratio formula
        # TODO: update scoring function to make it better
        # -> like use token count

        # example I was thinking about:
        letter_sum = sum(len(response) for response in system_output)
        ratio = compression_ratio(system_output) * min(1, len(system_output)/5) * min(1, letter_sum/100)
        # ratio = compression_ratio(system_output, algorithm='gzip',verbose=True)
        return EvaluationResult(is_correct=True,score=ratio,metadata = {'responses': system_output})


## Edit Distance (WIP)
- need to merge

In [74]:
%pip install -q python-Levenshtein

Note: you may need to restart the kernel to use updated packages.


In [1]:
import Levenshtein
from typing import List
from dataclasses import dataclass

@dataclass
class EvaluationResult:
    is_correct: bool
    score: float
    metadata: dict

class EditDistanceScoringEvaluator:

    def evaluate(self, system_output: List[str], **kwargs) -> EvaluationResult:
        if system_output is None or len(system_output) == 0:
            return EvaluationResult(is_correct=False, score=-1, metadata={})

        diversity_score = self.compute_distance(system_output)

        return EvaluationResult(
            is_correct=True, 
            score=diversity_score,
            metadata={'responses': system_output}
        )

    def compute_distance(self, outputs: List[str]) -> float:
        n = len(outputs)
        if n < 2:
            return 0.0

        total_distance = 0
        pairs = 0

        for i in range(n):
            for j in range(i + 1, n):
                dist = Levenshtein.distance(outputs[i], outputs[j])
                max_len = max(len(outputs[i]), len(outputs[j]))
                normalized_dist = dist / max_len if max_len > 0 else 0 
                total_distance += normalized_dist
                pairs += 1
        
        return total_distance / pairs if pairs > 0 else 0.0


In [2]:
distance_evaluator = EditDistanceScoringEvaluator()

# input_strs = [
#     ";lkjawefopajwiefpoij23jf9aj8sdfj8903jf908j -- Understanding the importance of effective communication in the workplace cannot be overstated. Clear communication fosters a positive environment where people can express their ideas and work together efficiently. When team members understand one another, they can collaborate seamlessly, avoid misunderstandings, and achieve collective goals. Furthermore, communication skills are essential for building trust, resolving conflicts, and ensuring that expectations are clear. Whether through verbal discussions, emails, or presentations, knowing how to convey thoughts in an understandable way is key to success in any professional setting.",
#     "fej89qw098efjq29f38j0938j20f398jqwe098fjq98wf -- In any workplace, the ability to communicate effectively is crucial for success. When individuals can clearly articulate their ideas and listen actively, it leads to a more productive and harmonious environment. Good communication prevents misunderstandings, aids in team collaboration, and helps in meeting shared objectives. It also plays a vital role in fostering trust among colleagues, resolving disputes, and ensuring transparency. Whether it’s through face-to-face conversations, written messages, or virtual meetings, mastering communication is essential to creating a positive, high-functioning work culture.",
#     "Effective communication is a cornerstone of a successful work environment. When employees communicate clearly and efficiently, it improves the overall flow of work and enhances collaboration. Clear exchanges of ideas help to eliminate confusion, build mutual trust, and ensure that everyone is aligned in their goals. Additionally, strong communication skills are key to managing conflicts and setting clear expectations among teams. Whether in meetings, emails, or other formats, being able to communicate effectively contributes to a thriving and efficient workplace.",
#     "The role of communication in the workplace cannot be overlooked. It serves as the foundation for successful teamwork and organizational growth. When team members share information clearly, it promotes a collaborative atmosphere and reduces the risk of errors or misinterpretations. Strong communication is also vital in building relationships, resolving issues, and making sure everyone is on the same page. Whether it's verbal exchanges or written correspondence, honing your ability to communicate well is vital for fostering an effective work environment.",
#     "Communication within the workplace is a vital element for success. Clear and open communication promotes a cooperative and efficient atmosphere, helping team members to better understand each other’s ideas and work toward common goals. It reduces confusion, builds trust, and allows for smoother problem-solving when conflicts arise. By conveying thoughts and expectations effectively, individuals can create stronger working relationships and a productive team dynamic. Whether through emails, phone calls, or face-to-face interactions, mastering communication techniques is key for professional achievement.",
# ]

input_strs = ["hi there", "hi", "hello", "yo whatup"]

# input_strs = ["This is a sample text with lots of repetition.", 
#                 "This is a sample text with lots of repetition.",
#                 "This is a sample text with lots of repetition."]

edit_distance = distance_evaluator.evaluate(input_strs)

print(f"Diversity Score: {edit_distance.score:.4f}")
print(f"Is Correct: {edit_distance.is_correct}")
print(f"Metadata: {edit_distance.metadata}")

Diversity Score: 0.8301
Is Correct: True
Metadata: {'responses': ['hi there', 'hi', 'hello', 'yo whatup']}


## Novelty Score
- need to merge

In [3]:
from typing import List
from dataclasses import dataclass
import numpy as np

@dataclass
class EvaluationResult:
    is_correct: bool
    score: float
    metadata: dict

class NoveltyScoringEvaluator:
    
    def evaluate(self, model: EmbeddingModel, system_output: List[str], **kwargs) -> EvaluationResult:
        if not system_output or len(system_output) == 0:
            return EvaluationResult(is_correct=False, score=-1, metadata={})

        novelty_scores = [self.compute_novelty(r, system_output[:i]) for i, r in enumerate(system_output)]

        avg_novelty = sum(novelty_scores) / len(novelty_scores) if novelty_scores else 0.0

        return EvaluationResult(
            is_correct=True,
            score=avg_novelty,
            metadata={'responses': system_output, 'novelty_scores': novelty_scores}
        )

    def compute_novelty(self, response: str, prior_responses: List[str]) -> float:
        if not prior_responses:
            return 1.0

        new_embedding = self.model.embed_text(response)
        prior_embeddings = [self.model.embed_text(r) for r in prior_responses]

        similarities = [
            np.dot(new_embedding, prior_embedding) /
            (np.linalg.norm(new_embedding) * np.linalg.norm(prior_embedding))
            for prior_embedding in prior_embeddings
        ]

        return 1 - max(similarities)


In [9]:
novelty_evaluator = NoveltyScoringEvaluator()

input_strs = ["hi there", "hi", "hello", "yo whatup"]

mock_model: MockEmbeddingModel = MockEmbeddingModel()
novelty = novelty_evaluator.evaluate(mock_model, input_strs)

print(f"Diversity Score: {novelty.score:.4f}")
print(f"Is Correct: {novelty.is_correct}")
print(f"Metadata: {novelty.metadata}")

EvaluationResult(is_correct=True, score=0.08368770360509659, metadata={'responses': ['Hello world!', 'Hi there!', 'Goodbye!']})


---
---

## Putting it all together

In [82]:
import numpy as np
mock_model: MockEmbeddingModel = MockEmbeddingModel()
cosine: CosineSimilarity = CosineSimilarity()
exact_evaluator = ExactMatchEvaluator()
diversity_evaluator = DiversityScoringEvaluator()
edit_dist_evaluator = EditDistanceScoringEvaluator()

def ensemble_diversity(strings):
    compression = diversity_evaluator.evaluate(strings)
    print("DiversityScoringEvaluator result:", compression)
    scores = list()
    for ind1 in range(len(strings)):
        ind2 = ind1+1 if ind1+1 != len(strings) else 0
        curr_score = calculate_text_similarity(text1=strings[ind1], text2=strings[ind2], model=mock_model, metric=cosine)
        print(f"SimilarityScore between ind1={ind1} and ind2={ind2}: {curr_score}")
        scores.append(curr_score)
    avg_score = np.average(scores)
    print(f"Avg cosine similarity: {avg_score}")
    print(f"diversity cosine-sim inverse: {1-avg_score}")
    edit_distance = edit_dist_evaluator.evaluate(strings)
    print(f"edit-dist score: {edit_distance.score:.4f}")
    print("-------------------------------")
    print(f"possible diversity score: {(1-avg_score) * compression.score * edit_distance.score}")

In [86]:
# input_strs = [
#     ";lkjawefopajwiefpoij23jf9aj8sdfj8903jf908j -- Understanding the importance of effective communication in the workplace cannot be overstated. Clear communication fosters a positive environment where people can express their ideas and work together efficiently. When team members understand one another, they can collaborate seamlessly, avoid misunderstandings, and achieve collective goals. Furthermore, communication skills are essential for building trust, resolving conflicts, and ensuring that expectations are clear. Whether through verbal discussions, emails, or presentations, knowing how to convey thoughts in an understandable way is key to success in any professional setting.",
#     "fej89qw098efjq29f38j0938j20f398jqwe098fjq98wf -- In any workplace, the ability to communicate effectively is crucial for success. When individuals can clearly articulate their ideas and listen actively, it leads to a more productive and harmonious environment. Good communication prevents misunderstandings, aids in team collaboration, and helps in meeting shared objectives. It also plays a vital role in fostering trust among colleagues, resolving disputes, and ensuring transparency. Whether it’s through face-to-face conversations, written messages, or virtual meetings, mastering communication is essential to creating a positive, high-functioning work culture.",
#     "Effective communication is a cornerstone of a successful work environment. When employees communicate clearly and efficiently, it improves the overall flow of work and enhances collaboration. Clear exchanges of ideas help to eliminate confusion, build mutual trust, and ensure that everyone is aligned in their goals. Additionally, strong communication skills are key to managing conflicts and setting clear expectations among teams. Whether in meetings, emails, or other formats, being able to communicate effectively contributes to a thriving and efficient workplace.",
#     "The role of communication in the workplace cannot be overlooked. It serves as the foundation for successful teamwork and organizational growth. When team members share information clearly, it promotes a collaborative atmosphere and reduces the risk of errors or misinterpretations. Strong communication is also vital in building relationships, resolving issues, and making sure everyone is on the same page. Whether it's verbal exchanges or written correspondence, honing your ability to communicate well is vital for fostering an effective work environment.",
#     "Communication within the workplace is a vital element for success. Clear and open communication promotes a cooperative and efficient atmosphere, helping team members to better understand each other’s ideas and work toward common goals. It reduces confusion, builds trust, and allows for smoother problem-solving when conflicts arise. By conveying thoughts and expectations effectively, individuals can create stronger working relationships and a productive team dynamic. Whether through emails, phone calls, or face-to-face interactions, mastering communication techniques is key for professional achievement.",
# ]

input_strs = ["hi there", "hi", "hello", "yo whatup"]

# input_strs = ["This is a sample text with lots of repetition.", 
#                 "This is a sample text with lots of repetition.",
#                 "This is a sample text with lots of repetition."]

ensemble_diversity(input_strs)

DiversityScoringEvaluator result: EvaluationResult(is_correct=True, score=0.063936, metadata={'responses': ['hi there', 'hi', 'hello', 'yo whatup']})
SimilarityScore between ind1=0 and ind2=1: 0.5207675658482732
SimilarityScore between ind1=1 and ind2=2: 0.6088947130341378
SimilarityScore between ind1=2 and ind2=3: 0.67913155770349
SimilarityScore between ind1=3 and ind2=0: 0.9344774636399475
Avg cosine similarity: 0.6858178250564622
diversity cosine-sim inverse: 0.31418217494353784
edit-dist score: 0.8301
-------------------------------
possible diversity score: 0.0166745277343434


In [97]:
num_jokes = 10
responses = []

for i in range(num_jokes):
    res = llm(prompt="Tell me a funny joke. Keep it concise.", model_id="openai:gpt-4o").data
    responses.append(res)
    print(f"Joke {i}: [{res}]")

print("-----")
ensemble_diversity(responses)

Joke 0: [Why don't scientists trust atoms?

Because they make up everything!]
Joke 1: [Why don't skeletons fight each other? They don't have the guts!]
Joke 2: [Why don't skeletons fight each other?

They don't have the guts.]
Joke 3: [Why don't scientists trust atoms?

Because they make up everything!]
Joke 4: [Why don't skeletons fight each other? They don't have the guts.]
Joke 5: [Why don't skeletons fight each other? They don't have the guts.]
Joke 6: [Why don't skeletons fight each other? They don't have the guts!]
Joke 7: [Why don’t skeletons fight each other? They don’t have the guts.]
Joke 8: [Why don't skeletons fight each other? They don't have the guts.]
Joke 9: [Why don't skeletons fight each other? They don't have the guts.]
-----
DiversityScoringEvaluator result: EvaluationResult(is_correct=True, score=3.45, metadata={'responses': ["Why don't scientists trust atoms?\n\nBecause they make up everything!", "Why don't skeletons fight each other? They don't have the guts!", "

In [98]:
prompts = 1
responses = []

for i in range(prompts):
    res = llm(prompt="Tell me 10 jokes. make them split with \'||\'. Don't say anything else besides the joke. ", model_id="openai:gpt-4o").data.split('||')
    responses += res

if prompts == 1 and len(responses) > 1:
    for i in range(len(responses)):
        print(f"Joke {i}: [{responses[i]}]")

print("-----")
ensemble_diversity(responses)

Joke 0: [Why don’t scientists trust atoms? Because they make up everything! ]
Joke 1: [ Parallel lines have so much in common. It’s a shame they’ll never meet. ]
Joke 2: [ Why did the scarecrow win an award? Because he was outstanding in his field! ]
Joke 3: [ I told my wife she was drawing her eyebrows too high. She looked surprised. ]
Joke 4: [ Why don’t skeletons fight each other? They don’t have the guts. ]
Joke 5: [ What do you call fake spaghetti? An impasta! ]
Joke 6: [ What’s brown and sticky? A stick! ]
Joke 7: [ Why was the math book sad? It had too many problems. ]
Joke 8: [ Can February March? No, but April May! ]
Joke 9: [ Why was the musician arrested? She got in treble.]
-----
DiversityScoringEvaluator result: EvaluationResult(is_correct=True, score=1.467, metadata={'responses': ['Why don’t scientists trust atoms? Because they make up everything! ', ' Parallel lines have so much in common. It’s a shame they’ll never meet. ', ' Why did the scarecrow win an award? Because 

---
---
## Improvements TODO
- Merge all functions
- fix ensembling
## Potential other cases to explore
- work ensembling all "diversity" related metrics 
  - add more metrics
  - tune added metrics
- combination of validation/hallucination metric + ensembled diversity metric -> score